In [ ]:

import matplotlib.pyplot as plt # 绘图
from matplotlib.ticker import MultipleLocator, MaxNLocator
import pandas as pd # 导出表格
from mpmath import mp, matrix, zeros, eye
from tqdm import tqdm

# 1.条件参数
time_array = list(range(720000,1+720000,1))
limited_height_list = list([1]) # 列出想看的限高情况，目前只有甘蔗和仙人掌，输入1,2就行了
harvesting_time = 4 # 收割造成的无法生长的时间，一般是粘性活塞粘液块加二档中继器为4tick；飞行器慢一些，算上往返应该是20tick。
harvest_array = list([10000]) # 不关心前几次收割的情况就只输入一万保证收割轮数多收敛就行了
公摊高度 = 4

标记图像极值点 = 1 # 填1或0代表是或否
picturePixelSize = 2160
pictureInchSize = picturePixelSize / 100

age_first = 0
age_last = 15

# 2.初始化与计算
产量表 = []
效率表 = []
体积效率表 = []

# 设置精度
mp.dps = 1000  # 设置为50位有效数字

# 假设这些变量已经定义：age_last, age_first, limited_height_list, harvest_array, time_array, harvesting_time

产量表 = []
效率表 = []
体积效率表 = []

for limited_height in limited_height_list:
    # 2.1.1 状态总数
    age_count = int(age_last - age_first + 1)
    states_count = int(age_count * limited_height + 1)

    # 2.1.2 初始状态矩阵 A
    A = zeros(1,states_count)
    A[0, states_count-1] = 1

    # 2.1.3 状态转移矩阵 T
    P = zeros(states_count,states_count)
    for i in range(states_count):
        P[i, i] = 1 - 1/4096
    for i in range(states_count - 1):
        P[i, i+1] = 1/4096
    P[states_count - 1, states_count - 1] = 1

    # 2.1.4 收割矩阵 H
    H = zeros(states_count,states_count)
    for i in range(age_count):
        H[i, i] = mp.mpf(1)
    for i in range(age_count, states_count):
        H[i, 0] = mp.mpf(1)

    # 2.2 遍历计算
    当前限高的产量表 = []
    当前限高的效率表 = []
    当前限高的体积效率表 = []
    for harvested_times in harvest_array:
        当前限高此轮收割的产量表 = []
        当前限高此轮收割的效率表 = []
        当前限高此轮收割的体积效率表 = []
        for t in time_array:
            P3t = P**(3 * t)
            S = A * ((P3t * H)**harvested_times) * P3t
            #print(S)
            product = mp.mpf(0)
            for height in range(limited_height):
                product += sum(S[(height * age_count):((height + 1) * age_count)]) * height
            product += limited_height * S[states_count - 1]
            产量 = float(product)
            效率 = float(product/(t+harvesting_time))
            体积效率 = float(product/(t+harvesting_time)/(limited_height + 公摊高度))
            print(f"限高{limited_height}, 收割{harvested_times}次, 时间{t}tick, 效率{效率}，体积效率{体积效率}，产量{产量}")

限高1, 收割10000次, 时间720000tick, 效率1.388881172882373e-06，体积效率2.7777623457647457e-07，产量1.388881172882373e-06
